In [192]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from compute_score import pearson_score, euclidean_score
import random

I imported all the necessary libraries:
- pandas to manage and read data files

In [193]:
df_movie = pd.read_csv('movie.csv')
df_rating = pd.read_csv('rating.csv')
df_tag = pd.read_csv('tag.csv')

In [194]:
print(df_movie.head())
print(df_rating.head())
print(df_tag.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
   userId  movieId            tag            timestamp
0      18     4141    Mark Waters  2009-04-24 18:19:40
1      

In [195]:
unique_movie = df_movie['movieId'].unique()
movie_id_name = {}
for m in unique_movie:
    movie_id_name[m] = (df_movie.loc[df_movie['movieId'] == m])['title']

In [196]:
unique_tag_list = []
for t in df_movie['genres']:
    temp = t.split('|')
    for word in temp:
        if word not in unique_tag_list:
            unique_tag_list.append(word)

In [197]:
genre = {}
for t in unique_tag_list:
    genre[t] = 0

In [198]:
print(genre)

{'Adventure': 0, 'Animation': 0, 'Children': 0, 'Comedy': 0, 'Fantasy': 0, 'Romance': 0, 'Drama': 0, 'Action': 0, 'Crime': 0, 'Thriller': 0, 'Horror': 0, 'Mystery': 0, 'Sci-Fi': 0, 'IMAX': 0, 'Documentary': 0, 'War': 0, 'Musical': 0, 'Western': 0, 'Film-Noir': 0, '(no genres listed)': 0}


In [199]:
X = {}
for m in unique_movie:
    X[str(m)] = genre.copy()

In [200]:
for movie in X:
    temp = str((df_movie.loc[df_movie['movieId'] == int(movie)])['genres']).split()[1]
    temp = temp.split('|')
    for g in temp:
        if g in X[movie]:
            X[movie][g] = 1

In [201]:
user_id = random.choice(df_rating['userId'].unique())
user_movies = df_rating.loc[df_rating['userId'] == user_id]
user_favorite_movie = user_movies.sort_values(by='rating', ascending=False).iloc[0]
user_favorite_movie_name = df_movie.loc[df_movie['movieId'] == user_favorite_movie['movieId']]['title'].to_string(index=False)
user_favorite_movie_rate = user_favorite_movie['rating']


In [202]:
p_score = {}
e_score = {}
print(user_favorite_movie)
for u in X:
    u_ = str(u)
    user_movie_ = str(user_favorite_movie['movieId'])
    if not u == user_id:
        p_score[u] = pearson_score(X, user_movie_, u_)
        e_score[u] = euclidean_score(X, user_movie_, u_)
print(p_score)
print(e_score)

userId                     46952
movieId                     1300
rating                       5.0
timestamp    1997-01-28 12:16:15
Name: 6823418, dtype: object
{'1': 0.19245008972987526, '2': -0.14002800840280097, '3': 0.4444444444444445, '4': 0.7934920476158722, '5': 0.6882472016116853, '6': -0.14002800840280097, '7': 0.4444444444444445, '8': -0.11111111111111112, '9': -0.07647191129018725, '10': -0.14002800840280097, '11': 0.7934920476158722, '12': 0.4444444444444445, '13': -0.14002800840280097, '14': 0.6882472016116853, '15': -0.14002800840280097, '16': 0.4444444444444445, '17': 0.4444444444444445, '18': 0.6882472016116853, '19': 0.6882472016116853, '20': 0.5773502691896257, '21': 0.3267320196065356, '22': 0.19245008972987526, '23': -0.14002800840280097, '24': 0.4444444444444445, '25': 0.4444444444444445, '26': 0.6882472016116853, '27': 0.4444444444444445, '28': 0.4444444444444445, '29': 0.19245008972987526, '30': 0.4444444444444445, '31': 0.6882472016116853, '32': -0.1400280084028

In [208]:
sorted_p_dic = {k: v for k, v in sorted(p_score.items(), key=lambda item: item[1], reverse=True)}
sorted_e_dic = {k: v for k, v in sorted(e_score.items(), key=lambda item: item[1], reverse=True)}

print(sorted_p_dic)
print(sorted_e_dic)

{'72': 1.0, '75': 1.0, '82': 1.0, '84': 1.0, '96': 1.0, '106': 1.0, '133': 1.0, '156': 1.0, '171': 1.0, '178': 1.0, '194': 1.0, '205': 1.0, '218': 1.0, '235': 1.0, '272': 1.0, '304': 1.0, '308': 1.0, '322': 1.0, '324': 1.0, '345': 1.0, '352': 1.0, '371': 1.0, '430': 1.0, '450': 1.0, '472': 1.0, '476': 1.0, '500': 1.0, '516': 1.0, '528': 1.0, '562': 1.0, '567': 1.0, '568': 1.0, '569': 1.0, '570': 1.0, '574': 1.0, '582': 1.0, '583': 1.0, '615': 1.0, '617': 1.0, '635': 1.0, '639': 1.0, '649': 1.0, '657': 1.0, '763': 1.0, '765': 1.0, '767': 1.0, '771': 1.0, '776': 1.0, '794': 1.0, '819': 1.0, '847': 1.0, '864': 1.0, '877': 1.0, '939': 1.0, '973': 1.0, '984': 1.0, '988': 1.0, '994': 1.0, '1042': 1.0, '1045': 1.0, '1060': 1.0, '1120': 1.0, '1133': 1.0, '1146': 1.0, '1162': 1.0, '1173': 1.0, '1184': 1.0, '1243': 1.0, '1268': 1.0, '1279': 1.0, '1292': 1.0, '1300': 1.0, '1318': 1.0, '1392': 1.0, '1395': 1.0, '1410': 1.0, '1446': 1.0, '1454': 1.0, '1467': 1.0, '1476': 1.0, '1482': 1.0, '1529': 1

In [211]:


print('User: ', user_id, '\'s favorite movie is ', user_favorite_movie_name, ' and rated ', user_favorite_movie_rate)
print('Recomended movies that are similar are: ')
n = 1
mix_recomendation = 5
for i in sorted_p_dic:
    m_name = str((df_movie.loc[df_movie['movieId'] == int(i)])['title'].to_string(index=False))
    print('Movie #' +  i + '. ', m_name, 'with pearson score = ', p_score[i], ', and euclidean score = ', e_score[i])
    if n >= 5:
        break
    n += 1

User:  46952 's favorite movie is  My Life as a Dog (Mitt liv som hund) (1985)  and rated  5.0
Recomended movies that are similar are: 
Movie #72.  Kicking and Screaming (1995) with pearson score =  1.0 , and euclidean score =  1.0
Movie #75.  Big Bully (1996) with pearson score =  1.0 , and euclidean score =  1.0
Movie #82.  Antonia's Line (Antonia) (1995) with pearson score =  1.0 , and euclidean score =  1.0
Movie #84.  Last Summer in the Hamptons (1995) with pearson score =  1.0 , and euclidean score =  1.0
Movie #96.  In the Bleak Midwinter (1995) with pearson score =  1.0 , and euclidean score =  1.0
